<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/Sentinel_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sentinelsat geopandas shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 751.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
from shapely.geometry import box
from datetime import date

# User credentials for Copernicus Open Access Hub
username = 'your_username'
password = 'your_password'

# Path to your shapefile
shapefile_path = 'path_to_your_shapefile.shp'

# Convert shapefile to GeoJSON for easier handling
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg=4326)  # Ensure CRS is WGS84
roi = gdf.unary_union  # Merge all geometries into a single polygon
bbox = roi.bounds  # Get bounding box of the ROI

# Create GeoJSON format from bounding box
geojson_roi = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [bbox[0], bbox[1]],
                    [bbox[0], bbox[3]],
                    [bbox[2], bbox[3]],
                    [bbox[2], bbox[1]],
                    [bbox[0], bbox[1]]
                ]]
            },
            "properties": {}
        }
    ]
}

# Convert GeoJSON to WKT format
footprint = geojson_to_wkt(geojson_roi)

# Initialize Sentinel API
api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

# Search for Sentinel-2 data
products = api.query(
    footprint,
    date=('20230101', '20231231'),  # Date range: adjust as needed
    platformname='Sentinel-2',
    processinglevel='Level-2A',
    cloudcoverpercentage=(0, 30)  # Filter by cloud coverage
)

# Convert the products to a GeoDataFrame
gdf_products = api.to_geodataframe(products)

# Sort by cloud coverage and take the best product
gdf_products_sorted = gdf_products.sort_values(['cloudcoverpercentage'], ascending=[True])

# Display available products
print(gdf_products_sorted[['title', 'beginposition', 'cloudcoverpercentage']])

# Download the best product
if not gdf_products_sorted.empty:
    product_id = gdf_products_sorted.iloc[0].uuid
    api.download(product_id, directory_path='sentinel_downloads')

    print(f"Downloaded: {gdf_products_sorted.iloc[0]['title']}")
else:
    print("No suitable products found for the given criteria.")
